### A Simple RNN

In [1]:
## Importing Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
## Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

N x 1 x 28 x 28

In [3]:
## Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [4]:
## Create RNN Model
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size,hidden_size,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)

    #Forward
    out,_ = self.rnn(x,h0)
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)
    return out


In [5]:
## Create GRU Model
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size,hidden_size,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)

    #Forward
    out,_ = self.gru(x,h0)
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)
    return out


In [6]:
## Create LSTM Model
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)

    #Forward
    out,_ = self.lstm(x,(h0,c0))
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)
    return out


In [7]:
## Load Data
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [8]:
## Initialize Network
model = RNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [9]:
## Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
## Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores,targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or Adam step size
    optimizer.step()

100%|██████████| 938/938 [00:09<00:00, 98.20it/s]


In [11]:
## Check Accuracy on training & test to see how good our model 
def check_accuracy(loader,model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x,y  in loader:
      x = x.to(device=device).squeeze(1)
      y = y.to(device=device)

      # Forward
      scores = model(x)
      #64x10
      _,predictions = scores.max(1) #(max values for 2nd dim)/will output indices
      
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0) #Size of 1st dimension

  model.train()
  return num_correct/num_samples

print(f'Accuracy on training data: {check_accuracy(train_loader, model)*100:.2f}')
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training data: 98.48
Accuracy on test set: 98.31
